<h1> Cryptage RSA</h1>

<h2> Position du problème</h2>

On propose de réaliser la méthode de cryptage due à Rivest, Shamir et Adleman (dite méthode RSA).
Cet algorithme date de 1977, il repose sur le fait que, si l'on multiplie ensemble deux
nombres premiers très grands ($p$ et $q$),
il est très difficile de redécomposer le résultat obtenu ($n=p.q$) en facteurs premiers ($p$ et $q$).

[Historiquement, $p$ était un nombre de 64 chiffres et $q$ un nombre de 65 chiffres.
En 1994, le nombre $n=p.q$ de 129 chiffres (RSA129) a pu être décomposé en $p$ et $q$
grâce à 1600 ordinateurs travaillant en parallèle pendant deux jours.]

La méthode RSA, dite à clef publique, est la suivante:

[Les 2 premières étapes sont communes à toutes les méthodes de codage.]

1. à chaque lettre $\ell$ de l'alphabet, on affecte un nombre $n_\ell$ compris entre 1 et $n_{max}$.<br>
(Si on ne prend que les lettres majuscules non accentuées et le caractère blanc, on a $n_{max}=27$).<br>
<br>
2. on transforme le texte à transmettre (formé de $\ell_{max}$ lettres) en un nombre: %<br>
$\displaystyle message=\sum_{\ell=1}^{\ell_{max}}n_{\ell}\times n_{max}^\ell$.<br>
<br>
3. un interlocuteur $1$ choisit secrètement deux nombres premiers très grands $p_1$ et $q_1$ congrus à 2 modulo 3  [i.e. dont le reste de la division euclidienne par 3 est 2]. Il publie leur produit (la clef publique): $n_1=p_1.q_1$. Il garde secrètement le résultat d'un autre calcul (la clef privée):
$k_1={2.(p_1-1).(q_1-1)+1\over 3}$ ($k_1$ est forcément un entier).<br>
La fonction codage de $x$ consiste à calculer une certaine fonction $f_{n_1}(x)$;<br>
la fonction décodage de $x$ consiste à calculer la fonction inverse: $f^{-1}_{n_1,k_1}(x)$.<br>
<br>
4. un interlocuteur $2$ fait de même avec $p_2$, $q_2$, $n_2=p_2.q_2$ et $k_2={2.(p_2-1).(q_2-1)+1\over 3}$.<br>
<br>
5. lorsque 1 veut transmettre le $message$ à 2, il envoie: $xxxx=f_{n_2}(f^{-1}_{n_1,k_1}(message))$.<br>
<br>
6. lorsque 2 veut décoder $xxxx$, il calcule $f_{n_1}(f^{-1}_{k_2,n_2}(xxxx))=message$.<br>
<br>
7. le récepteur décompose le nombre $message$ dans la base $n_{max}$ et
transforme chaque nombre de la décompo\-sition en lettre de l'alphabet pour retrouver le texte.
<br>

La fonction codage $f_{n_i}(yyyy)$ consiste à décomposer $yyyy$ dans la base $n_i$ puis
à élever chaque terme de la décomposition au cube modulo $n_i$.

La fonction décodage $f^{-1}_{n_i,k_i}(zzzz)$ consiste à élever $zzzz$ à la puissance $3.k_i$ modulo $n_i$.
Cette procédure utilise le petit théorème de Fermat généralisé:<br>
- soit $m$ et $n$ deux nombres premiers entre eux ($m<n$);<br>
- soit $\Phi(n)$ la fonction d'Euler donnant le nombre des entiers inférieurs et premiers avec $n$.<br>
Ici $\Phi(n)=(p-1)*(q-1)$.<br>
On a alors: $m^{\Phi(n)}\equiv1$ $[n]$ $\Rightarrow$ $m^{(p-1).(q-1)}\equiv1$ $[n]$
$\Rightarrow$ $\forall k\in\mathbb{N}$, $m^{k.(p-1).(q-1)}\equiv1$ $[n]$<br>
$\Rightarrow$ $\forall k\in\mathbb{N}$, $m^{k.(p-1).(q-1)+1}\equiv m$ $[n]$<br>
Comme $p\equiv q\equiv 2$ $[3]$, on a: $2.(p-1).(q-1)+1\equiv 0$ $[3]$
et donc $\exists k\in\mathbb{N}$ tel que $2.(p-1).(q-1)+1=3.k$

Utilisez le programme <b>Rabin-Miller</b> pour vérifier si un nombre est premier.

In [2]:
open Random;;
Random.self_init;;

- : unit -> unit = <fun>

In [3]:
#load "nums.cma";;
let zero = Big_int.big_int_of_int 0;;
let un = Big_int.big_int_of_int 1;;
let deux = Big_int.big_int_of_int 2;;
let trois = Big_int.big_int_of_int 3;;
let cinq = Big_int.big_int_of_int 5;;

val zero : Big_int.big_int = <abstr>

val un : Big_int.big_int = <abstr>

val deux : Big_int.big_int = <abstr>

val trois : Big_int.big_int = <abstr>

val cinq : Big_int.big_int = <abstr>

In [4]:
let rec pow x n =
    if n=0 then 1 else
    if n mod 2 = 0 then pow (x*x) (n/2)
    else x*(pow (x*x) (n/2));;
    
let charsize = Big_int.big_int_of_int (pow 2 (8*4));; (* Unicode : jusqu'à quatre octets par caractère *)
let code s =
    let rec loop i num decalage =
        if i = String.length s then num
        else loop (i+1) (Big_int.add_big_int num (Big_int.mult_int_big_int (int_of_char s.[i]) decalage)) (Big_int.mult_big_int decalage charsize) in
    loop 0 zero un;;

let decode n =
    let rec loop n s =
        if Big_int.eq_big_int n zero then s else 
        begin
           let c = String.make 1 (char_of_int(Big_int.int_of_big_int (Big_int.mod_big_int n charsize))) in
           loop (Big_int.div_big_int n charsize) (s^c);
        end in
    loop n "";;

val pow : int -> int -> int = <fun>

val charsize : Big_int.big_int = <abstr>

val code : string -> Big_int.big_int = <fun>

val decode : Big_int.big_int -> string = <fun>

In [5]:
let temoin_Miller n a =
    (* determination de s et d tels que n-1 = 2**s * d *)
    let s= ref 0 in
    let m= ref (Big_int.pred_big_int n) in
    while (Big_int.eq_big_int (Big_int.mod_big_int !m deux) zero) do
        incr s;
        m := Big_int.div_big_int !m deux;
    done;
    let d = ref !m in
    let a_ = ref a in
    
    (* calcul de x = a**d % n *)
    let x= ref un in
    while Big_int.gt_big_int !d zero do
        if not (Big_int.eq_big_int (Big_int.mod_big_int !d deux) zero) then
            x := Big_int.mod_big_int (Big_int.mult_big_int !x !a_) n;
        a_ := Big_int.mod_big_int (Big_int.mult_big_int !a_ !a_) n;
        d := Big_int.div_big_int !d deux;
    done;
   
    if Big_int.eq_big_int !x un ||  Big_int.eq_big_int !x (Big_int.pred_big_int n) then false
    else begin
    
    let continue = ref true in
    while !continue && !s>1 do
        x := Big_int.mod_big_int (Big_int.mult_big_int !x !x) n;
        if  Big_int.eq_big_int !x (Big_int.pred_big_int n) then
            continue := false;
        decr s;
    done;
    !continue;
    end;;
let max_rand_int = int_of_float (2.**30.);;
let test_Miller_Rabin (n:Big_int.big_int) k =
    if Big_int.eq_big_int n deux 
    || Big_int.eq_big_int n trois
    || Big_int.eq_big_int n cinq
    then true
    else begin
        let trouve = ref true in
        let i = ref 0 in 
        let a = ref deux in
        while !trouve && !i<k do
           if Big_int.lt_big_int n (Big_int.big_int_of_int max_rand_int) then
              a:= Big_int.big_int_of_int (2+ Random.int (Big_int.int_of_big_int n-2)) (* entre 2 et n-1 inclus *)
           else
              a:= Big_int.big_int_of_int (2+ Random.int (max_rand_int-2));
          if temoin_Miller n !a then trouve := false;
          incr i;
        done; !trouve;
    end;;
let isprime n = test_Miller_Rabin n 10;;

val temoin_Miller : Big_int.big_int -> Big_int.big_int -> bool = <fun>

val max_rand_int : int = 1073741824

val test_Miller_Rabin : Big_int.big_int -> int -> bool = <fun>

val isprime : Big_int.big_int -> bool = <fun>

In [6]:
let rec primes n =
    let prime = Big_int.big_int_of_int(pow 10 n + (Random.int (pow 10 (n+1)))) in
    if Big_int.eq_big_int (Big_int.mod_big_int prime trois) deux && isprime prime then prime
    else primes n;;

val primes : int -> Big_int.big_int = <fun>

In [7]:
let print_big_int i = print_string (Big_int.string_of_big_int i)

val print_big_int : Big_int.big_int -> unit = <fun>

In [8]:
let liste_en_nombre liste cle =
    let rec loop liste res decalage =
        match liste with
        | [] -> res
        | hd::tl -> loop tl (Big_int.add_big_int res (Big_int.mult_big_int hd decalage)) (Big_int.mult_big_int decalage cle) in
    loop liste zero un;;

val liste_en_nombre :
 Big_int.big_int list -> Big_int.big_int -> Big_int.big_int = <fun>

In [9]:
let codage nombre cle =
    Big_int.mod_big_int (Big_int.power_big_int_positive_int nombre 3) cle;;

val codage : Big_int.big_int -> Big_int.big_int -> Big_int.big_int = <fun>

In [10]:
let decomposition_sur_base nombre base =
    let rec loop nombre res =
        if Big_int.eq_big_int nombre zero then List.rev res
        else loop (Big_int.div_big_int nombre base) ((Big_int.mod_big_int nombre base)::res) in
    loop nombre [];;

val decomposition_sur_base :
 Big_int.big_int -> Big_int.big_int -> Big_int.big_int list = <fun>

In [11]:
List.map print_big_int (decomposition_sur_base (Big_int.big_int_of_int 123) (Big_int.big_int_of_int 10))

321

- : unit list = [(); (); ()]

In [12]:
let rsa nombre cle_publique =
    let liste_decodee=decomposition_sur_base nombre cle_publique in
    let liste_codee = List.map (fun nombre -> codage nombre cle_publique) liste_decodee in
    liste_en_nombre liste_codee cle_publique;;

val rsa : Big_int.big_int -> Big_int.big_int -> Big_int.big_int = <fun>

<h4>Modular exponentiation (wikipedia)</h4>
The algorithm is as follows:

    Set c = 1, e′ = 0.
    Increase e′ by 1.
    Set c = (b ⋅ c) mod m.
    If e′ < e, goto step 2. Else, c contains the correct solution to c ≡ be mod m.


In [13]:
let power_mod b e m =
    let rec loop e' c =
        if Big_int.ge_big_int e' e then c
        else begin
           let c_ = Big_int.mod_big_int (Big_int.mult_big_int b c) m in
           let e_ = Big_int.add_big_int e' un in
           loop e_ c_
           end in
    loop zero un;;

val power_mod :
 Big_int.big_int -> Big_int.big_int -> Big_int.big_int -> Big_int.big_int =
 <fun>

Test <tt>power_mod</tt>

In [14]:
print_big_int(power_mod (Big_int.big_int_of_int 4) (Big_int.big_int_of_int 13) (Big_int.big_int_of_int 497))

445

- : unit = ()

<h4>Exponentiation modulaire (wikipedia)</h4>

    Bignum modpow(Bignum base, Bignum exp, Bignum m) {

       Bignum result = 1;

       while (exp > 0) {
          if (exp & 1 > 0) result = (result * base) % m;
          exp >>= 1;
          base = (base * base) % m;
       }

       return result;

     }

In [15]:
let power_mod base exponentiel modulo =
    let rec loop b e res =
        if Big_int.eq_big_int e zero then res
        else if Big_int.eq_big_int (Big_int.mod_big_int e deux) un then
        (* e impair *)
        loop (Big_int.mod_big_int (Big_int.mult_big_int b b) modulo)
             (Big_int.div_big_int e deux)
             (Big_int.mod_big_int (Big_int.mult_big_int res b) modulo)
        else
        loop (Big_int.mod_big_int (Big_int.mult_big_int b b) modulo)
             (Big_int.div_big_int e deux)
             res
        in
    loop base exponentiel un;;

val power_mod :
 Big_int.big_int -> Big_int.big_int -> Big_int.big_int -> Big_int.big_int =
 <fun>

Test <tt>power_mod</tt>

In [16]:
print_big_int(power_mod (Big_int.big_int_of_int 4) (Big_int.big_int_of_int 13) (Big_int.big_int_of_int 497))

445

- : unit = ()

In [17]:
let x = Big_int.big_int_of_string "2988348162058574136915891421498819466320163312926952423791023078876139" in
let n = Big_int.big_int_of_string "2351399303373464486466122544523690094744975233415544072992656881240319" in
let m = Big_int.big_int_of_string "10000000000000000000000000000000000000000" in
print_big_int (power_mod x n m)

1527229998585248450016808958343740453059

- : unit = ()

In [18]:
(*Decodage*)
let decodage nombre cle_privee cle_publique =
    power_mod nombre cle_privee cle_publique;;

val decodage :
 Big_int.big_int -> Big_int.big_int -> Big_int.big_int -> Big_int.big_int =
 <fun>

In [19]:
let rsa_ nombre cle_privee cle_publique =
    let liste_codee = decomposition_sur_base nombre cle_publique in
    let liste_decodee = List.map (fun nombre -> decodage nombre cle_privee cle_publique) liste_codee in
    liste_en_nombre liste_decodee cle_publique;;

val rsa_ :
 Big_int.big_int -> Big_int.big_int -> Big_int.big_int -> Big_int.big_int =
 <fun>

In [20]:
print_string "longueur de la clef : ";;
let n = 5;;
let p = primes (n-1);;
print_string ("Recherche de nombres premiers à "^(string_of_int n)^" chiffres...");;

longueur de la clef : Recherche de nombres premiers à 5 chiffres...

- : unit = ()

val n : int = 5

val p : Big_int.big_int = <abstr>

- : unit = ()

In [21]:
let p1 = primes (n-1);;
let q1 = primes (n-1);;
(* Pour les tests: p1=350381; q1=350411 *)
let p1 = Big_int.big_int_of_int 350381;;
let q1 = Big_int.big_int_of_int 350411;;
let p1 = Big_int.big_int_of_int 2709147011;;
let q1 = Big_int.big_int_of_int 1308592931;;
let n1 = Big_int.mult_big_int p1 q1;;
let k1 = Big_int.div_big_int (Big_int.succ_big_int (Big_int.mult_big_int deux 
        (Big_int.mult_big_int (Big_int.pred_big_int p1) (Big_int.pred_big_int q1)))) trois ;;
print_string "p1 = ";print_big_int p1;
print_string ", q1 = ";print_big_int q1;
print_string ", n1 = ";print_big_int n1;
print_string ", k1 = ";print_big_int k1;;

p1 = 2709147011, q1 = 1308592931, n1 = 3545170627634379241, k1 = 2363447082411092867

val p1 : Big_int.big_int = <abstr>

val q1 : Big_int.big_int = <abstr>

val p1 : Big_int.big_int = <abstr>

val q1 : Big_int.big_int = <abstr>

val p1 : Big_int.big_int = <abstr>

val q1 : Big_int.big_int = <abstr>

val n1 : Big_int.big_int = <abstr>

val k1 : Big_int.big_int = <abstr>

- : unit = ()

In [22]:
let p2 = primes (n-1);;
let q2 = primes (n-1);;
(* Pour les tests: p2=350423; q2=350429 *)
let p2 = Big_int.big_int_of_int 350423;;
let q2 = Big_int.big_int_of_int 350429;;
let p2 = Big_int.big_int_of_int 6195912131;;
let q2 = Big_int.big_int_of_int 9193973189;;
let n2 = Big_int.mult_big_int p2 q2;;
let k2 = Big_int.div_big_int (Big_int.succ_big_int (Big_int.mult_big_int deux 
        (Big_int.mult_big_int (Big_int.pred_big_int p2) (Big_int.pred_big_int q2)))) trois ;;
print_string "p2 = ";print_big_int p2;
print_string ", q2 = ";print_big_int q2;
print_string ", n2 = ";print_big_int n2;
print_string ", k2 = ";print_big_int k2;;

p2 = 6195912131, q2 = 9193973189, n2 = 56965050013813855759, k2 = 37976699998949313627

val p2 : Big_int.big_int = <abstr>

val q2 : Big_int.big_int = <abstr>

val p2 : Big_int.big_int = <abstr>

val q2 : Big_int.big_int = <abstr>

val p2 : Big_int.big_int = <abstr>

val q2 : Big_int.big_int = <abstr>

val n2 : Big_int.big_int = <abstr>

val k2 : Big_int.big_int = <abstr>

- : unit = ()

In [23]:
print_string("\n Test\n");;
(* 1 envoie un message 2 *)
print_string "Message chiffré : "; print_big_int(code "Bonjour, ca va ?");;


 Test
Message chiffré : 196670158693166349770915387796196729055219905386382323134976706254773804848967582950484123344065386602546442942740951267627043359897844426932748354

- : unit = ()

- : unit = ()

In [24]:
let rsa_ nombre cle_privee cle_publique =
    let liste_codee = decomposition_sur_base nombre cle_publique in
    let liste_decodee = List.map (fun nombre -> decodage nombre cle_privee cle_publique) liste_codee in
    liste_en_nombre liste_decodee cle_publique;;

val rsa_ :
 Big_int.big_int -> Big_int.big_int -> Big_int.big_int -> Big_int.big_int =
 <fun>

In [25]:
print_string "Message chiffré->ASR: ";print_big_int(rsa_(code "Bonjour, ca va ?") k1 n1);;

Message chiffré->ASR: 9461356373189678172094580799560455558558642228169352797766750654332308572823694882456995650541174767422612226869180353585624335130255073853009983678

- : unit = ()

In [26]:
let message_envoye=rsa (rsa_(code "Bonjour, ca va ?") k1 n1) n2;;
print_string "message envoyé = chiffré->ASR->RSA: ";print_big_int message_envoye;

message envoyé = chiffré->ASR->RSA: 21985293836931802086875545043191818710996051950106323297928188447073029071912692360128600482359104291721400309513686140251045675656415276105717633711260634289

val message_envoye : Big_int.big_int = <abstr>

- : unit = ()

In [27]:
(*2 décode le message de 1*)
print_string "message recu->ASR: ";print_big_int (rsa_ message_envoye k2 n2)

message recu->ASR: 9461356373189678172094580799560455558558642228169352797766750654332308572823694882456995650541174767422612226869180353585624335130255073853009983678

- : unit = ()

In [28]:
let message_recu=rsa(rsa_ message_envoye k2 n2) n1;;
print_string "message recu->ASR->RSA->déchiffré:";print_string( decode message_recu );;

message recu->ASR->RSA->déchiffré:Bonjour, ca va ?

val message_recu : Big_int.big_int = <abstr>

- : unit = ()

In [39]:
(*3 connait n1 et n2 mais pas k2, il ne peut pas décoder. Exemple:*)
let p3=Big_int.big_int_of_int 350447;; (* Prime[30008]*)
let q3=Big_int.big_int_of_int 350453;; (* Prime[30009]*)
let k3=Big_int.div_big_int (Big_int.succ_big_int (Big_int.mult_big_int deux 
        (Big_int.mult_big_int (Big_int.pred_big_int p3) (Big_int.pred_big_int q3)))) trois ;;

val p3 : Big_int.big_int = <abstr>

val q3 : Big_int.big_int = <abstr>

val k3 : Big_int.big_int = <abstr>

In [41]:
decode(rsa(rsa_ message_envoye k3 n2) n1) (*si on exécute cette ligne -> bug car code ascii hors texte*)

Exception: Invalid_argument "char_of_int".

In [35]:
let message = "Hello world !";;
let enc = rsa( rsa_ (code message) k1 n1) n2;;
let dec = decode(rsa(rsa_ enc k2 n2) n1);;
print_string "Message encodé : ";print_big_int enc;;
print_string ("\nMessage décodé : "^dec);;

Message encodé : 1834845884299802914688022969567746365839596440800798927453590405157863306194615717291967063219827599619341674458821371403028598482338511222
Message décodé : Hello world !

val message : string = "Hello world !"

val enc : Big_int.big_int = <abstr>

val dec : string = "Hello world !"

- : unit = ()

- : unit = ()